# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.109914e+02     1.517198e+00
 * time: 0.6551768779754639
     1     1.075656e+01     9.181533e-01
 * time: 1.8612239360809326
     2    -1.135157e+01     1.017175e+00
 * time: 2.397350788116455
     3    -3.395202e+01     8.069783e-01
 * time: 3.226378917694092
     4    -4.743429e+01     6.276000e-01
 * time: 3.9999380111694336
     5    -5.705116e+01     2.227474e-01
 * time: 4.8019139766693115
     6    -5.988743e+01     1.551811e-01
 * time: 5.442120790481567
     7    -6.095469e+01     4.583492e-02
 * time: 5.972431898117065
     8    -6.137436e+01     4.955922e-02
 * time: 6.516424894332886
     9    -6.163572e+01     3.784033e-02
 * time: 7.063226938247681
    10    -6.184561e+01     3.077059e-02
 * time: 7.601094007492065
    11    -6.200704e+01     1.883761e-02
 * time: 8.536406993865967
    12    -6.208497e+01     1.411549e-02
 * time: 9.06083083152771
    13    -6.213049e+01     1.329343e-02
 * time: 9.581295013427734
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671076
    AtomicLocal         -18.8557693
    AtomicNonlocal      14.8522648
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485378 
    Xc                  -19.3336823

    total               -62.261666457984